In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_excel('../data/pmii.xlsx', header=1)

In [4]:
df.shape

(8189, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8189 entries, 0 to 8188
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Tanggal              8189 non-null   datetime64[ns]
 1   IN / OUT             8189 non-null   object        
 2   Nomor GDN/GRN        8188 non-null   object        
 3   Gudang               8189 non-null   object        
 4   Dari/Kepada-Masking  8189 non-null   object        
 5   Nama Barang          8189 non-null   object        
 6   Merk/Jenis-Deleted   4344 non-null   object        
 7   Satuan               8187 non-null   object        
 8   Masuk                812 non-null    float64       
 9   Keluar               7374 non-null   float64       
 10  Nilai                7672 non-null   float64       
 11  Jumlah               7763 non-null   float64       
 12  Berat                8094 non-null   object        
 13  Total Berat          8189 non-nul

In [7]:
(df
 .isna()
 .sum()
)

Tanggal                   0
IN / OUT                  0
Nomor GDN/GRN             1
Gudang                    0
Dari/Kepada-Masking       0
Nama Barang               0
Merk/Jenis-Deleted     3845
Satuan                    2
Masuk                  7377
Keluar                  815
Nilai                   517
Jumlah                  426
Berat                    95
Total Berat               0
dtype: int64

## CLEAN

1. fillna Masuk with 0
2. fillna Keluar with 0
3. Merk fill with "unknown"
4. Satuan fill with "-"
5. Nilai fill with 0
6. Berat fill with 0 
7. jumlah fillna 0 and astype integer
8. columns replace space with "_" replace "/" with "OR"

In [5]:
# change columns names
df.columns = [i.replace("/", "or").replace(" ", "_") if "IN" in i else i.replace(" ", "_").replace("-", "_").replace("/", "_atau_") for i in df.columns.tolist()]

In [6]:
def fill_na(dataframe):
    cat= ['Masuk', 'Keluar', 'Nilai', 'Berat', 'Jumlah']
    
    
    iss = (dataframe['Jumlah'].isna()) & (dataframe['Nilai'].notna())
    
    dataframe.loc[iss, :].fillna(dataframe.Masuk*dataframe.Nilai, inplace=True)
    
    #fill with 0
    for i in cat:
        dataframe[i] = dataframe[i].fillna(0)
    
    #fill with unknown and -
    dataframe.Merk_atau_Jenis_Deleted.fillna("unknown", inplace=True)
    dataframe.Satuan.fillna("-", inplace=True)
    dataframe.Nomor_GDN_atau_GRN.fillna("-", inplace=True)
    return dataframe
    
def replace_(data):
    ls = ['Sample', '#ERROR!', 'Unloading', 'Lain-lain', 'Bocor']
    for i in ls:
        data.Dari_atau_Kepada_Masking.replace(i, "Others", inplace=True)
    
(df
    .pipe(fill_na)
    .pipe(replace_)
)

In [8]:
category = {
    'Obat': ['Obat', 'Vitamin', 'Hemaviton', 'Suplemen'],
    'Healthcare': ['Sanitizer', 'Disinfektan', 'Hygiene', 'Hansaplast', 'Termometer', 'Rapid', 'USG', 'Pembalut', 'Tisu', 'Spray', 'Sabun', 'Telon', 'Soap', 'Gigi', 'Shampo', 'Family'],
    'Apparel': ['Jas', 'Rompi', 'Tshirt', 'Underware', 'Handuk', 'Sandal', 'Handuk', 'Sejadah', 'Mukena', 'Pakaian', 'Apron', 'Cover', 'Head Cap', 'Perlengkapan Bayi', 'Sarung', 'Clothes', 'Shirt', 'Kemeja'],
    'Alat Pembersih': ['Virkon', 'Wipol', 'Kaporit', 'Detergen', 'Sunlight', 'Biosol', 'Hit Aerosol'],
    'Atk & Cetakan': ['Poster', 'Stiker', 'Tripod', 'Banner', 'Baliho', 'Pamflet', 'Leaflet', 'Form', 'Buku', 'CTPS'],
    'Perlengkapan Tidur': ['Matras', 'Tikar', 'Selimut', 'Bantal', 'Sprei'],
    'Alat Evakuasi': ['Boat', 'Life', 'Dayung', 'Helm', 'River'],
    'Alat Pelindung Diri': ['Masker', 'Hazmat', 'Boot', 'Sarung', 'Mayat', 'Shield', 'Kacamata'],
    'HVAC': ['Ventilator'],
    'Others': ['Terpal', 'Tandon', 'Kompresor', 'PHBS', 'Jerigen', 'Knapsack', 'Dovie', 'Assortment', 'Wastafel', 'Selang', 'Karyawan', 'Lebaran', 'Samsung', 'Boneka', 'Pisau', 'Paket steps kids']
    }

def cate(a):
    return [(j, i[0]) for i in a.items() for j in i[1]]

In [9]:
def generelize(series, match_name, default):
    seen = None
    for match, name in match_name:
        mask = series.str.contains(match)
        if seen is None:
            seen = mask
        else:
            seen =  seen | mask #equal with seen |= mask
        series = series.where(~mask, name)
    series = series.where(seen, default) # set deafult other cathegory
    return series

data = (df
            .assign(
                 Expired_OR_Reject = lambda df_: generelize(df_['Dari_atau_Kepada_Masking'], 
                                                         [("Expired", "Yes"), ("Tidak Layak", "Yes")], "No"),
                Kategori = lambda df_: generelize(df_['Nama_Barang'], 
                                                         cate(category), "Makanan"
                    )
            .replace({"Expired (Dimusnahkan)":"-", "Tidak Layak digunakan (Reject)":"-"})
           )
       )
        

In [11]:
def unique_id(data):    
    data = data.assign(
                    categori_id = (data
                       .groupby(by=['Kategori'])
                       .ngroup()
                          ),
                    item_id = (data
                       .groupby(by=['Kategori', 'Nama_Barang'])
                       .ngroup()
                          )
                )
    a=2
    for i in ['categori_id', 'item_id']:
        data[i] = data[i].apply(lambda x: str(x).zfill(a))
        a += 1
    
    data =(data
            .assign(unique_id=data
                                .categori_id
                                .str
                                .cat(data.item_id, sep='-')
                                .str
                                .cat(data.Gudang)
                   )
             .drop(columns=['categori_id', 'item_id'])
            )
    return data

In [12]:
qwe = (data
     .pipe(unique_id)
)

In [13]:
qwe

,Tanggal,IN_or_OUT,Nomor_GDN_atau_GRN,Gudang,Dari_atau_Kepada_Masking,Nama_Barang,Merk_atau_Jenis_Deleted,Satuan,Masuk,Keluar,Nilai,Jumlah,Berat,Total_Berat,Expired_OR_Reject,Kategori,unique_id
0,2020-02-01,IN,01.01,G2,Gudang Regional PMI_Serang,Masker Bedah,Sensi,Pcs,350000.0,0.0,7000.0,2.450000e+09,0.006,2100.00,No,Alat Pelindung Diri,01-010G2
1,2020-02-01,OUT,01.01,G2,Ketua Umum,Masker N95,3M,Pcs,0.0,800.0,108240.0,8.659200e+07,0.04,32.00,No,Alat Pelindung Diri,01-012G2
2,2020-02-01,OUT,01.02,G2,PMI Kabupaten Natuna,Masker Bedah,Sensi,Pcs,0.0,20000.0,7000.0,1.400000e+08,0.006,120.00,No,Alat Pelindung Diri,01-010G2
3,2020-02-05,OUT,05.01,G2,Palang Merah Hongkong,Masker Bedah,Sensi,Pcs,0.0,50000.0,7000.0,3.500000e+08,0.006,300.00,No,Alat Pelindung Diri,01-010G2
4,2020-02-05,OUT,05.02,G2,Others,Masker Bedah,Sensi,Pcs,0.0,4000.0,7000.0,2.800000e+07,0.006,24.00,No,Alat Pelindung Diri,01-010G2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,2021-04-26,OUT,-,G4,Tidak Layak digunakan (Reject),Kopi Kapal Api,unknown,Pcs,0.0,10.0,0.0,0.000000e+00,0,0.00,Yes,Makanan,07-134G4
8185,2021-04-26,OUT,-,G4,Tidak Layak digunakan (Reject),Mie Instan,unknown,Pcs,0.0,9.0,0.0,0.000000e+00,0,0.00,Yes,Makanan,07-138G4
8186,2021-04-26,OUT,-,G4,Tidak Layak digunakan (Reject),Cheetos Snack 15 g,unknown,Pcs,0.0,46.0,0.0,0.000000e+00,0,0.00,Yes,Makanan,07-119G4
8187,2021-04-27,OUT,27.01,G4,Staff MP PMI,Wipol,unknown,Pouch,0.0,2.0,15500.0,3.100000e+04,0.78,1.56,No,Alat Pembersih,02-025G4


In [14]:
sdf=(qwe
        .set_index('Tanggal')
        .groupby('unique_id',dropna=False)
        .resample('M')
        .Keluar
        .sum()
        .unstack()
        .copy()
    )

# add Total columns
sdf['Total']=(sdf
        .fillna(0)
        .sum(axis=1)
    )

# add days columns
sdf['Days']=(sdf
        .iloc[:, :-1]
        .count(axis=1)
        .mul(30)
    )

# add Daily_Usage columns
sdf=(sdf
    .assign(Daily_Usage=(sdf
                            .Total
                            .div(sdf.Days)
                            .round(0)
                        )
            
            )
            
)

# add lead time columns
sdf['Lead_Time'] =7

# add Safety_Stock columns
sdf=(sdf
    .assign(Safety_Stock=(sdf
                            .Daily_Usage
                            .mul(sdf.Lead_Time)
                            .mul(2)
            
                        )
            )
)

# add Reorder_Point columns
sdf=(sdf
    .assign(Reorder_Point=((sdf
                            .Daily_Usage
                            .mul(sdf.Lead_Time))
                            + sdf.Safety_Stock
            
                        ),
            Reorder_Quantity=(sdf
                                .Daily_Usage
                                .mul(sdf.Lead_Time)
                            )
            )
)

# add Max_Stock_Quantity
sdf=(sdf
    .assign(Max_Stock_Quantity=(sdf
                            .Reorder_Point
                            .mul(2)
            
                        )
            )
)

In [23]:
cob = sdf.loc[:, 'Safety_Stock':'Max_Stock_Quantity'].copy()

In [26]:
huft = (qwe
            .merge(cob, left_on='unique_id', right_index=True, how='outer')
            .rename(columns={'Total':'Total_Quantity'})
       )

In [32]:
asd = (qwe
     .groupby('unique_id')[['Masuk','Keluar']]
     .sum()
)

fun=(asd
     .mask(asd.Masuk==0, asd.Keluar, axis=0)
     .merge(qwe, left_index=True, right_on='unique_id', suffixes=('_SUM', '_NOSUM'))
)

fun=fun.sort_index()

In [34]:
def turnover(data):
    persediaan_awal = 0
#     barang_masuk = masuk
#     barang_keluar = keluar
    jumlah_hari_pengamatan = 451
    
    data = data.assign(persediaan_akhir=(persediaan_awal+data.Masuk_SUM)-data.Keluar_SUM)
    data = data.assign(persediaan_rata2=(persediaan_awal+data.persediaan_akhir)/2)
    data = data.assign(turn_over_ratio_partial=(data.Keluar_SUM / data.persediaan_rata2))
    data.turn_over_ratio_partial = data.turn_over_ratio_partial.mask(data.turn_over_ratio_partial==np.inf, 0)
    data = data.assign(lama_waktu_penyimpanan=(jumlah_hari_pengamatan / data.turn_over_ratio_partial))
    data.lama_waktu_penyimpanan = data.lama_waktu_penyimpanan.mask(data.lama_waktu_penyimpanan==np.inf, 0)
    data = data.assign(turn_over_ratio_annually=(365 / data.lama_waktu_penyimpanan))
    data.turn_over_ratio_annually = data.turn_over_ratio_annually.mask(data.turn_over_ratio_annually==np.inf, 0)
    
    return data

In [35]:
fuu = fun.pipe(turnover)

In [36]:
fuu

,Masuk_SUM,Keluar_SUM,Tanggal,IN_or_OUT,Nomor_GDN_atau_GRN,Gudang,Dari_atau_Kepada_Masking,Nama_Barang,Merk_atau_Jenis_Deleted,Satuan,Masuk_NOSUM,Keluar_NOSUM,Nilai,Jumlah,Berat,Total_Berat,Expired_OR_Reject,Kategori,unique_id,persediaan_akhir,persediaan_rata2,turn_over_ratio_partial,lama_waktu_penyimpanan,turn_over_ratio_annually
0,2809020.0,2558320.0,2020-02-01,IN,01.01,G2,Gudang Regional PMI_Serang,Masker Bedah,Sensi,Pcs,350000.0,0.0,7000.0,2.450000e+09,0.006,2100.00,No,Alat Pelindung Diri,01-010G2,250700.0,125350.0,20.409414,22.097646,16.517596
1,72625.0,49626.0,2020-02-01,OUT,01.01,G2,Ketua Umum,Masker N95,3M,Pcs,0.0,800.0,108240.0,8.659200e+07,0.04,32.00,No,Alat Pelindung Diri,01-012G2,22999.0,11499.5,4.315492,104.507204,3.492582
2,2809020.0,2558320.0,2020-02-01,OUT,01.02,G2,PMI Kabupaten Natuna,Masker Bedah,Sensi,Pcs,0.0,20000.0,7000.0,1.400000e+08,0.006,120.00,No,Alat Pelindung Diri,01-010G2,250700.0,125350.0,20.409414,22.097646,16.517596
3,2809020.0,2558320.0,2020-02-05,OUT,05.01,G2,Palang Merah Hongkong,Masker Bedah,Sensi,Pcs,0.0,50000.0,7000.0,3.500000e+08,0.006,300.00,No,Alat Pelindung Diri,01-010G2,250700.0,125350.0,20.409414,22.097646,16.517596
4,2809020.0,2558320.0,2020-02-05,OUT,05.02,G2,Others,Masker Bedah,Sensi,Pcs,0.0,4000.0,7000.0,2.800000e+07,0.006,24.00,No,Alat Pelindung Diri,01-010G2,250700.0,125350.0,20.409414,22.097646,16.517596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,10.0,10.0,2021-04-26,OUT,-,G4,Tidak Layak digunakan (Reject),Kopi Kapal Api,unknown,Pcs,0.0,10.0,0.0,0.000000e+00,0,0.00,Yes,Makanan,07-134G4,0.0,0.0,0.000000,0.000000,0.000000
8185,9.0,9.0,2021-04-26,OUT,-,G4,Tidak Layak digunakan (Reject),Mie Instan,unknown,Pcs,0.0,9.0,0.0,0.000000e+00,0,0.00,Yes,Makanan,07-138G4,0.0,0.0,0.000000,0.000000,0.000000
8186,116.0,116.0,2021-04-26,OUT,-,G4,Tidak Layak digunakan (Reject),Cheetos Snack 15 g,unknown,Pcs,0.0,46.0,0.0,0.000000e+00,0,0.00,Yes,Makanan,07-119G4,0.0,0.0,0.000000,0.000000,0.000000
8187,2426.0,2426.0,2021-04-27,OUT,27.01,G4,Staff MP PMI,Wipol,unknown,Pouch,0.0,2.0,15500.0,3.100000e+04,0.78,1.56,No,Alat Pembersih,02-025G4,0.0,0.0,0.000000,0.000000,0.000000


In [47]:
def fastslow(masuk, keluar, tor):
    if (masuk > 0 and keluar == 0) or (tor < 1 and tor > 0):
        tor =  "Non-Moving"
    elif masuk == keluar or tor < 0:
        tor =  "-"
    elif tor >= 1 and tor <= 3:
        tor =  "Slow Moving"
    elif tor > 3:
        tor = "Fast Moving"
    return tor

fuu['fast_slow'] = fuu.apply(lambda x: fastslow(x.Masuk_SUM, x.Keluar_SUM, x.turn_over_ratio_annually),axis=1)

In [50]:
fuu = fuu.sort_index().drop(columns=['Masuk_SUM', 'Keluar_SUM'])

In [51]:
fy = fuu[['unique_id','turn_over_ratio_annually','fast_slow']]

In [64]:
cv = pd.read_excel("D:/project PMI/pmi final.xlsx")

In [69]:
cv[['Umur_Barang_(Hari)', 'Masa_Kadaluarsa_(Hari)', 'Sisa_Umur_Barang_(Hari)']]=cv[['Umur_Barang_(Hari)', 'Masa_Kadaluarsa_(Hari)', 'Sisa_Umur_Barang_(Hari)']].fillna(0)

In [76]:
ambil = cv.loc[:, ['unique_id','Current_Stock_Qty', 'Umur_Barang_(Hari)','Masa_Kadaluarsa_(Hari)','Sisa_Umur_Barang_(Hari)']]

,unique_id,Current_Stock_Qty,Umur_Barang_(Hari),Masa_Kadaluarsa_(Hari),Sisa_Umur_Barang_(Hari)
0,01-010G2,250700,59.0,1440.0,1381.0
1,01-012G2,22999,65.0,1440.0,1375.0
2,01-010G2,250700,59.0,1440.0,1381.0
3,01-010G2,250700,59.0,1440.0,1381.0
4,01-010G2,250700,59.0,1440.0,1381.0
...,...,...,...,...,...
8184,07-134G4,0,0.0,0.0,0.0
8185,07-138G4,0,0.0,0.0,0.0
8186,07-119G4,0,0.0,0.0,0.0
8187,02-025G4,0,0.0,0.0,0.0


In [56]:
juuu = huft.merge(fy, left_index=True, right_index=True).drop(columns='unique_id_y').rename(columns={'unique_id_x':'unique_id'})

In [81]:
fix = juuu.merge(ambil, left_index=True, right_index=True).drop(columns='unique_id_y').rename(columns={'unique_id_x':'unique_id'})